First, import needed modules

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML
import os

Initalize variables

In [44]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [45]:
data = pd.read_csv(f'{path}/data/ltw.csv', encoding = 'UTF-8-SIG', sep=',')
data.head(10)

,AGS,GEN,year,Land,post_2010,post_2014,Union,SPD,Grüne,FDP,...,avg_income,avg_age,age 0-17,age 18-24,age 25-44,age 45-64,age 65+,protestant,catholic,religion_other
0,3101000,Braunschweig,2008.0,NI,0.0,0.0,36.861503,31.245464,11.023763,8.087549,...,31800.0,43.6,14.6,9.2,28.6,26.7,20.9,0.395453,0.140082,0.464465
1,3101000,Braunschweig,2013.0,NI,1.0,0.0,29.666789,33.671272,18.608499,8.250600,...,31800.0,43.6,14.6,9.2,28.6,26.7,20.9,0.395453,0.140082,0.464465
2,3101000,Braunschweig,2017.0,NI,1.0,1.0,26.342641,37.604703,11.862375,7.904115,...,31800.0,43.6,14.6,9.2,28.6,26.7,20.9,0.395453,0.140082,0.464465
3,3102000,Salzgitter,2008.0,NI,0.0,0.0,37.476063,39.268958,4.291938,5.740138,...,26200.0,45.5,16.4,7.8,21.9,30.1,23.8,0.404833,0.176531,0.418636
4,3102000,Salzgitter,2013.0,NI,1.0,0.0,30.880127,44.075623,9.252751,6.053921,...,26200.0,45.5,16.4,7.8,21.9,30.1,23.8,0.404833,0.176531,0.418636
5,3102000,Salzgitter,2017.0,NI,1.0,1.0,24.445995,45.203436,4.266019,5.319891,...,26200.0,45.5,16.4,7.8,21.9,30.1,23.8,0.404833,0.176531,0.418636
6,3103000,Wolfsburg,2008.0,NI,0.0,0.0,43.121141,31.515784,6.509262,6.691886,...,31000.0,44.6,15.3,8.1,26.2,27.8,22.7,0.402120,0.172331,0.425548
7,3103000,Wolfsburg,2013.0,NI,1.0,0.0,37.629895,35.812910,10.901908,7.342017,...,31000.0,44.6,15.3,8.1,26.2,27.8,22.7,0.402120,0.172331,0.425548
8,3103000,Wolfsburg,2017.0,NI,1.0,1.0,29.942107,43.211115,5.754535,6.092242,...,31000.0,44.6,15.3,8.1,26.2,27.8,22.7,0.402120,0.172331,0.425548
9,3151001,Adenbüttel,2008.0,NI,0.0,0.0,47.568209,30.960854,6.287070,7.117438,...,42300.0,40.7,22.2,7.4,23.2,32.2,14.9,0.587834,0.096646,0.315520


Statsmodels

DiD for treatment 50 in 2014

In [46]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ treated_50*post_2014 + Land', data=data).fit(cov_type='cluster', cov_kwds={'groups': np.array(data[['AGS', 'year']])}) #locals converts string to variable name
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Primary Vote, Municipalities within 50 km of a powerline project, treatment year 2013')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
with open(f'{path}/tables/ltw/ltw_treatment50.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())

DiD for treatment 50 in 2014 with controls

In [48]:
# first drop na values
ctrl_data = data.dropna(axis='rows', how='any', subset=['pop_density', 'male', 'female', 'foreign', 'employed', 'unemployed', 'avg_income', 'avg_age', 'protestant', 'catholic', 'religion_other'])

In [57]:
controls = 'pop_density + male + foreign + unemployed + np.log(avg_income) + avg_age + catholic'
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ treated_50*post_2014 + Land + {controls}', data=ctrl_data).fit(cov_type='cluster', cov_kwds={'groups': np.array(ctrl_data[['AGS', 'year']])}) #locals converts string to variable name
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Primary Vote, Municipalities within 50 km of a powerline project, treatment year 2013')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
with open(f'{path}/tables/ltw/ltw_treatment50_controls.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())

DiD for treatment 50 in 2010

In [ ]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    locals()[party] = smf.ols(formula=f'{party} ~ treated_50*post_2010 + Land', data=data).fit(cov_type='cluster', cov_kwds={'groups': np.array(data[['AGS', 'year']])}) #locals converts string to variable name
stargazer = Stargazer([Union, SPD, FDP, Linke, Grüne, Andere])
stargazer.title('Effect on Primary Vote, Municipalities within 50 km of a powerline project, treatment year 2013')
stargazer.custom_columns(['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'], [1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
with open(f'{path}/tables/ltw/ltw_treatment50.tex', 'w') as f:
    f.write(stargazer.render_latex())
HTML(stargazer.render_html())